# DeepStart data genertion

This program creates training data for deepstar usng the dijkstra algorithm on a randomly generated perlin noise heightmap.

## Import python modules

In [1]:
import os
import random
import noise
import math
import time
import matplotlib as plt
import numpy as np
from PIL import Image
from IPython.display import clear_output

## Data generation options

#### Data settings
**Shape**: Image dimensions in pixels.<br>
**Output**: Output folder path for images.<br>
**Images**: How many images to generate.<br>

#### Noise settings
**Scale**: Noise scale.<br>
**Octaves**: How many times to apply the noise.<br>
**Persistence**: How much the noise amplitue changes for each octave.<br>
**Lacunarity**: How much the noise frequency changes for each octave.

In [2]:
shape = (256, 256)
output = "data/"
images = 1

scale = 100.0
octaves = 6
persistence = 0.5
lacunarity = 2.0

## Node class

Class to keep information used by the dijkstra algorithm. The move cost return how much it will cost to move from this node to another. Now it is calculated as the height difference between the two pluss a small constant to make sure it will take the shortest path along a flat plane.

In [3]:
class Node:
    def __init__(self,value,point):
        self.value = value
        self.dist = 999999
        self.point = point
        self.parent = None
        self.H = 0
        self.G = 0

    def move_cost(self,other):
        if self.value > other.value:
            return self.value - other.value + 0.01
        else:
            return other.value - self.value + 0.01

## Define helper functions

**Children**: Returns all neighbour nodes for any given nodes.<br>
**Get Height**: Returns the perlin noise at (i, j) with the given settings.<br>
**Pretty Print**: Display progress information in a visually pleasing way.

In [4]:
def children(point,grid):
    x,y = point.point
    liste = []
    if x-1 > 0:
        liste.append((x-1, y))
    if x+1 < shape[0]:
        liste.append((x+1, y))
    if y-1 > 0:
        liste.append((x, y-1))
    if y+1 < shape[1]:
        liste.append((x, y+1))

    return [grid[d[0]][d[1]] for d in liste]

def get_height(i, j):
    return noise.pnoise2(i/scale, j/scale, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=shape[0], repeaty=shape[1], base=0)

def pretty_print(images, progress):
    line_len = 41 + len(str(images))
    progress_line_len = 24
    bar_len = line_len - progress_line_len + 1

    percent = int(progress * 100);
    
    print("=" * line_len)
    print(f'== Starting data generation of {images} image{("s" if images != 1 else "")} ==')
    print("=" * line_len)
    print(f'== Progress: {" " * (2 - len(str(percent)))}{percent}% == {"#" * math.floor(bar_len * progress)}{" " * math.ceil(bar_len * (1 - progress))} ==')
    print("=" * line_len)

## Dijkstra algorithim implementation

Dijkstra is the algorithm used for this data generation. Here is it explained via psuedo code from wikipedia.

In [5]:
def djikstra(start, goal, grid):
    start.dist = 0

    openset = set()

    visited = {}
    current = start
    while True:
        if current == goal:
            path = []
            while current.parent:
                path.append(current)
                current = current.parent

            return path

        for n in children(current, grid):
            if (not visited.__contains__(n)):
                openset.add(n)
                if n.dist > current.dist + n.move_cost(current):
                    n.dist = current.dist + n.move_cost(current)
                    n.parent = current

        if (len(openset) == 0):
            return
                    
        visited[current] = current
        current = min(openset, key=lambda n: n.dist)
        openset.remove(current)

## Data generation

In [6]:
start = (random.randrange(2, shape[1] - 2), random.randrange(2, shape[1] - 2))
end = (random.randrange(2, shape[1] - 2), random.randrange(2, shape[1] - 2))
prev_progress = 0

pretty_print(images, 0)
for image in range(images):
    im = Image.new("RGB", shape, "#00FF00")
    pixels = im.load()

    world = []
    for i in range(shape[0]):
        world.append([])
        for j in range(shape[1]):
            n = get_height(i, j)
            n += 1
            n /= 2

            world[i].append(Node(n, (i, j)))

    path = djikstra(world[start[0]][start[1]], world[end[0]][end[1]], world)
    if path is None:
        continue
    
    midpoint = path[math.floor(len(path) / 2)]

    for i in range(shape[0]):
        for j in range(shape[1]):
            intNoise = int(world[i][j].value * 255)
            pixels[i, j] = (intNoise, 0, 0)

    im.paste((0, 255, 0), (start[0] -1, start[1] -1, start[0], start[1]))
    im.paste((0, 0, 255), (end[0] -1, end[1] -1, end[0], end[1]))
    im.save(output + "{0}_{1}_confirm.png".format(midpoint.point[0], midpoint.point[1], image))

    progress = round(((image + 1) / images), 2)
    if (progress != prev_progress):
        clear_output(wait=False)
        pretty_print(images, progress)
        prev_progress = progress

    start = (random.randrange(0, shape[1] - 0), random.randrange(0, shape[1] - 0))
    end = (random.randrange(0, shape[1] - 0), random.randrange(0, shape[1] - 0))

== Starting data generation of 1 image ==
== Progress: 100% == ################### ==
